In [1]:
import pandas as pd

In [2]:
crime_features = pd.read_csv('/Users/nateoppenheimer/code/willbanny/Location-Analysis/crime_trial_bigQuery.csv')

In [3]:
google_features = pd.read_csv('/Users/nateoppenheimer/code/willbanny/Location-Analysis/google_trail_bigQuery.csv')

In [4]:
dep_df = pd.read_csv('/Users/nateoppenheimer/code/willbanny/Location-Analysis/raw_data/deprivation_raw.csv')

In [5]:
grid_df = pd.read_csv('/Users/nateoppenheimer/code/willbanny/Location-Analysis/raw_data/features/bquxjob_2efec792_188918eb4d5.csv')

In [6]:
dep_df = dep_df[dep_df['Local_Authority_District_name__2019_']=='Leicester']

In [7]:
engineered_dep = grid_df.copy()

In [8]:
from scipy import spatial

In [9]:
row_list =[]
for index, rows in dep_df.iterrows():
    my_list =(rows.longitude, rows.latitude)
    row_list.append(my_list)

In [10]:
tree = spatial.KDTree(row_list)

In [11]:
def ret_long_lat_key(c1):
    return row_list[tree.query([(c1['Longitude'],c1['Latitude'])])[1][0]]

In [12]:
coor_index = grid_df.apply(lambda c1: ret_long_lat_key(c1), axis=1)
coor_index

0      (-1.171282200228854, 52.67160519568661)
1       (-1.15725941431731, 52.67830469005258)
2      (-1.139418313841044, 52.66667919455223)
3       (-1.15725941431731, 52.67830469005258)
4      (-1.171282200228854, 52.67160519568661)
                        ...                   
95    (-1.118561383618473, 52.598353073761274)
96     (-1.154785577439872, 52.58327306907491)
97     (-1.154785577439872, 52.58327306907491)
98      (-1.162826983614924, 52.5940662765374)
99    (-1.1491532403855007, 52.58964341032872)
Length: 100, dtype: object

In [13]:
engineered_dep['lng'] = coor_index.map(lambda x: x[0])
engineered_dep['lat'] = coor_index.map(lambda x: x[1])
# engineered_dep

In [14]:
up_df = pd.merge(engineered_dep, dep_df,  how='left', left_on=['lng','lat'], right_on = ['longitude','latitude'])

In [15]:
up_df = up_df.drop(columns=['lng', 'lat'])

In [16]:
up_df.head(1)

,District,Description,POLYGON_ID,GridName,Longitude,Latitude,int64_field_0,lsoa11cd,LSOA_name__2011_,Local_Authority_District_code__2019_,...,Outdoors_Sub_domain_Score,Outdoors_Sub_domain_Rank__where_1_is_most_deprived_,Outdoors_Sub_domain_Decile__where_1_is_most_deprived_10__of_LSOAs_,Total_population__mid_2015__excluding_prisoners_,Dependent_Children_aged_0_15__mid_2015__excluding_prisoners_,Population_aged_16_59__mid_2015__excluding_prisoners_,Older_population_aged_60_and_over__mid_2015__excluding_prisoners_,Working_age_population_18_59_64__for_use_with_Employment_Deprivation_Domain__excluding_prisoners__,longitude,latitude
0,City of Leicester (B),Unitary Authority,97712,Grid2_0,-1.178881,52.687005,13210,E01013618,Leicester 001C,E06000016,...,0.292,10864,4,1575,296,971,308,994,-1.171282,52.671605


In [17]:
up_df['Index_of_Multiple_Deprivation__IMD__Score']

0     27.035
1     12.917
2     47.497
3     12.917
4     27.035
       ...  
95    11.235
96    51.943
97    51.943
98    34.751
99    58.904
Name: Index_of_Multiple_Deprivation__IMD__Score, Length: 100, dtype: float64

In [18]:
up_df['longitude']

0    -1.171282
1    -1.157259
2    -1.139418
3    -1.157259
4    -1.171282
        ...   
95   -1.118561
96   -1.154786
97   -1.154786
98   -1.162827
99   -1.149153
Name: longitude, Length: 100, dtype: float64

In [53]:
from sklearn.neighbors import BallTree
import numpy as np
import pandas as pd

In [54]:
bt = BallTree(np.deg2rad(dep_df[['latitude', 'longitude']].values), metric='haversine')

distances, indices = bt.query(np.deg2rad(grid_df[['Latitude', 'Longitude']]))

In [55]:
indices.shape

(100, 1)

In [56]:
good = dep_df.iloc[indices[:, 0]]

In [57]:
grid_df[['Longitude', 'Latitude']]

,Longitude,Latitude
0,-1.178881,52.687005
1,-1.164038,52.687005
2,-1.134352,52.687005
3,-1.149195,52.687005
4,-1.178888,52.678007
...,...,...
95,-1.119727,52.588025
96,-1.164166,52.579026
97,-1.149359,52.579026
98,-1.178964,52.588025


In [59]:
good[['lng', 'lat']] = grid_df[['Longitude', 'Latitude']].values

/var/folders/1b/mzc5rqk96mx4bl0ywffcsc0r0000gn/T/ipykernel_97799/2894072045.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good[['lng', 'lat']] = grid_df[['Longitude', 'Latitude']].values
/var/folders/1b/mzc5rqk96mx4bl0ywffcsc0r0000gn/T/ipykernel_97799/2894072045.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good[['lng', 'lat']] = grid_df[['Longitude', 'Latitude']].values


In [60]:
good

,int64_field_0,lsoa11cd,LSOA_name__2011_,Local_Authority_District_code__2019_,Local_Authority_District_name__2019_,Index_of_Multiple_Deprivation__IMD__Score,Index_of_Multiple_Deprivation__IMD__Rank__where_1_is_most_deprived_,Index_of_Multiple_Deprivation__IMD__Decile__where_1_is_most_deprived_10__of_LSOAs_,Income_Score__rate_,Income_Rank__where_1_is_most_deprived_,...,Outdoors_Sub_domain_Decile__where_1_is_most_deprived_10__of_LSOAs_,Total_population__mid_2015__excluding_prisoners_,Dependent_Children_aged_0_15__mid_2015__excluding_prisoners_,Population_aged_16_59__mid_2015__excluding_prisoners_,Older_population_aged_60_and_over__mid_2015__excluding_prisoners_,Working_age_population_18_59_64__for_use_with_Employment_Deprivation_Domain__excluding_prisoners__,longitude,latitude,lng,lat
21279,13207,E01013615,Leicester 001A,E06000016,Leicester,12.917,21145,7,0.070,21444,...,4,1520,301,1003,216,1004,-1.157259,52.678305,-1.178881,52.687005
21279,13207,E01013615,Leicester 001A,E06000016,Leicester,12.917,21145,7,0.070,21444,...,4,1520,301,1003,216,1004,-1.157259,52.678305,-1.164038,52.687005
21279,13207,E01013615,Leicester 001A,E06000016,Leicester,12.917,21145,7,0.070,21444,...,4,1520,301,1003,216,1004,-1.157259,52.678305,-1.134352,52.687005
21279,13207,E01013615,Leicester 001A,E06000016,Leicester,12.917,21145,7,0.070,21444,...,4,1520,301,1003,216,1004,-1.157259,52.678305,-1.149195,52.687005
8165,13210,E01013618,Leicester 001C,E06000016,Leicester,27.035,9568,3,0.179,8074,...,4,1575,296,971,308,994,-1.171282,52.671605,-1.178888,52.678007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,13269,E01013679,Leicester 035B,E06000016,Leicester,55.718,1386,1,0.336,1282,...,3,1536,482,893,161,847,-1.131030,52.595230,-1.119727,52.588025
1963,13267,E01013677,Leicester 036E,E06000016,Leicester,51.943,1879,1,0.283,2774,...,5,1491,348,851,292,840,-1.154786,52.583273,-1.164166,52.579026
1963,13267,E01013677,Leicester 036E,E06000016,Leicester,51.943,1879,1,0.283,2774,...,5,1491,348,851,292,840,-1.154786,52.583273,-1.149359,52.579026
6178,13203,E01013611,Leicester 034D,E06000016,Leicester,34.751,5980,2,0.167,8995,...,3,1569,310,833,426,829,-1.162827,52.594066,-1.178964,52.588025
